# Data Loading and fature extraction
below you will see that we load data with three techniques each of them will work with one of the three models

Loading Data from Kaggle

In [1]:
!pip install -q kaggle 
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d kameo4189/aflw2000-300wlp

100% 3.64G/3.65G [00:30<00:00, 161MB/s]
100% 3.65G/3.65G [00:30<00:00, 128MB/s]


In [3]:
%%capture
!pip install mediapipe

In [4]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow
import glob

In [5]:
%%capture
!unzip /content/aflw2000-300wlp.zip

In [6]:
def load_data(load_type='whole'):
  data = dict()
  faceModule = mediapipe.solutions.face_mesh
  with faceModule.FaceMesh(static_image_mode=True) as faces:
    for img in glob.glob("/content/AFLW2000-3D/AFLW2000/*.jpg"):
        # loading the image
        image = cv2.imread(img)
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
          # looping over the faces in the image
          face = results.multi_face_landmarks[0]
          for i,landmark in enumerate(face.landmark):
            # nose is the index 1 in landmarks
              dist = np.sqrt((face.landmark[152].x-face.landmark[10].x)**2 + (face.landmark[152].y-face.landmark[10].y)**2)
              x = (landmark.x - face.landmark[1].x) / dist
              y = (landmark.y - face.landmark[1].y) / dist
              if load_type == 'whole':
                  data.setdefault(str(i)+'_x',[])
                  data[str(i)+'_x'].append(x)
                  data.setdefault(str(i)+'_y',[])
                  data[str(i)+'_y'].append(y)
              elif load_type == 'avg':
                  data.setdefault(str(i),[])
                  data[str(i)].append((x*0.7+y*0.3))
              elif load_type == 'angle':
                  dx = x - face.landmark[1].x
                  dy = y - face.landmark[1].y
                  theta = math.atan2(dy, np.sqrt(dx*dx))
                  data.setdefault(str(i),[])
                  data[str(i)].append(theta)
              else: return None
          mat_file = sio.loadmat('mat'.join(img.split('jpg')))
          # extracting the labels 3 angels
          pose_para = mat_file["Pose_Para"][0][:3]
          pitch = pose_para[0]
          yaw = pose_para[1]
          roll = pose_para[2]
          data.setdefault('pitch',[])
          data['pitch'].append(pitch)
          data.setdefault('yaw',[])
          data['yaw'].append(yaw)   
          data.setdefault('roll',[])
          data['roll'].append(roll) 
  return data   

Load ALL features with normalization by centeralizing all points around the nose and divide them by the face distance Data most approbriate for pitch model

In [7]:
data_df = pd.DataFrame(load_data('whole'))
data_df.to_csv('scaled_features.csv',index=False)

In [8]:
data_df

,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,4_y,...,464_y,465_x,465_y,466_x,466_y,467_x,467_y,pitch,yaw,roll
0,-0.053593,0.102101,0.0,0.0,-0.021508,0.026599,0.008365,-0.141758,0.014760,-0.037808,...,-0.205444,0.121536,-0.200303,0.334412,-0.172664,0.353935,-0.177299,-0.194714,-0.048253,0.354045
1,0.022561,0.094589,0.0,0.0,0.000708,0.029606,-0.087753,-0.114346,-0.014363,-0.037534,...,-0.246298,-0.052590,-0.235446,0.118441,-0.357333,0.131741,-0.374950,-0.199571,-0.170665,-0.389337
2,0.066344,0.117143,0.0,0.0,0.051398,0.034609,-0.003382,-0.129524,-0.006200,-0.038839,...,-0.239629,0.138409,-0.230410,0.383019,-0.288985,0.407868,-0.307670,-0.052379,0.456169,-0.082544
3,-0.031546,0.149034,0.0,0.0,-0.056459,0.055877,-0.090930,-0.112135,-0.002308,-0.039888,...,-0.192680,-0.117549,-0.187167,-0.030418,-0.227918,-0.026292,-0.236849,0.158786,-0.704738,-0.216958
4,-0.009297,0.103847,0.0,0.0,-0.011022,0.032257,-0.050500,-0.124920,-0.002272,-0.039254,...,-0.230305,0.011924,-0.221025,0.187750,-0.284305,0.203703,-0.299941,-0.071097,-0.175177,-0.112254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,-0.088769,0.128823,0.0,0.0,-0.097662,0.046268,-0.074970,-0.120413,0.006763,-0.040041,...,-0.230782,-0.137736,-0.218428,-0.130820,-0.263684,-0.132448,-0.280637,0.265343,-1.172513,-0.194743
1849,0.033318,0.128235,0.0,0.0,0.055700,0.057922,0.069399,-0.122443,0.007482,-0.039011,...,-0.164555,0.257483,-0.162298,0.477783,-0.109414,0.515737,-0.126584,0.489906,1.215544,0.490315
1850,-0.000177,0.134134,0.0,0.0,0.035418,0.052005,0.061467,-0.124429,0.011516,-0.037883,...,-0.150952,0.251267,-0.151118,0.514018,-0.080638,0.548699,-0.087679,0.021257,0.493857,0.395608
1851,-0.040666,0.126698,0.0,0.0,-0.035678,0.039582,-0.039801,-0.130512,0.007445,-0.039534,...,-0.207563,0.009580,-0.203211,0.179638,-0.217223,0.197103,-0.226273,-0.055964,-0.377750,0.104433


Extract features with normalization by centeralizing all points around the nose and divide them by the face distance and instead of recording each x, y coordinate of the feature i will record average of them
by understanding the problem and after alot of trials this average is the best for modeling yaw

In [9]:
data_compressed_df = pd.DataFrame(load_data('avg'))
data_compressed_df.to_csv('scaled_features_avgComposed.csv',index=False)      

Extract contours features with normalization by centeralizing all points around the nose and divide them by the face distance and instead of recording each x, y coordinate of the feature i will record the ange between each point and the noise.

this angle is the best calculation for roll

In [10]:
data_compArctan_df = pd.DataFrame(load_data('angle'))
data_compArctan_df.to_csv('scaled_features_arctanComposed.csv',index=False)    